## Notebook: Implementing and optimizing the forward-pass of a convolutional layer using CUDA

### 1. Hướng dẫn chạy Notebook:


+ Bọn em sẽ gửi cho thầy một thư mục chứa toàn bộ những gì có trong đồ án ở trên Google drive và github
+ Ở đây, thì bọn em chạy trên colab với Hardware accelerator: T4 GPU
+ Sau khi thầy tải toàn bộ thư mục, thì thầy sẽ lấy thư mục source_code và up lên Google drive để có thể thực hiện chạy code.
+ Lưu ý: Ở dưới có bước %cd, nên thầy cần phải cd vào được trong thư mục source code (Cú pháp được viết rõ ở dưới ở cell có #Lưu ý)
+ Sau đó thì thầy chỉ cần cell !make, đợi và chạy cell !./demo là có thể nhận được kết quả
+ Thầy có thể make clean để xóa toàn bộ file.o và file demo được tạo và **chỉnh sửa đôi chút trong code** để có thể chạy được các cài đặt khác
+ Về việc **chỉnh sửa đôi chút trong code**: Thì thầy chỉ cần nhấn vào **conv.cu** thay đổi biến **use_kernel** trong hàm

 **void Conv::forward(const Matrix& bottom)**

Cụ thể việc thay đổi là:

// 0: Kernel 0

// 1: Kernel 1

// 2: Kernel 2

// Other: Host

(Thầy có thể xem presentation video để xem rõ hơn về cách chạy)

### 2. Chạy code:

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Lưu ý: %cd /content/drive/???/source_code
# Với ??? là thư mục thầy tạo đưa thư mục source_code vô
# Lưu ý:  %cd /content/drive/source_code
# Nếu thầy upload thẳng thư mục source_code lên
%cd /content/drive/MyDrive/Parallel_Programming_Final_Project/source_code

/content/drive/MyDrive/Parallel_Programming_Final_Project/source_code


#### 2.1. Cài đặt tuần tự

In [5]:
!make

gcc   -c -o src/network.o src/network.cc -Ithird_party/eigen
gcc   -c -o src/mnist.o src/mnist.cc -Ithird_party/eigen
gcc   -c -o src/layer/fully_connected.o src/layer/fully_connected.cc -Ithird_party/eigen
gcc   -c -o src/layer/max_pooling.o src/layer/max_pooling.cc -Ithird_party/eigen
gcc   -c -o src/layer/relu.o src/layer/relu.cc -Ithird_party/eigen
gcc   -c -o src/layer/softmax.o src/layer/softmax.cc -Ithird_party/eigen
nvcc   -c -o src/layer/conv.o src/layer/conv.cu -Ithird_party/eigen
third_party/eigen/Eigen/src/Core/DenseStorage.h(223): warning #20012-D: __host__ annotation is ignored on a function("DenseStorage") that is explicitly defaulted on its first declaration
      __attribute__((host)) __attribute__((device)) DenseStorage(const DenseStorage&) = default;
                     ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

third_party/eigen/Eigen/src/Core/DenseStorage.h(223): warning #20012-D: __device__ annotation is ignored on a functio

In [6]:
!./demo

mnist train number: 60000
mnist test number: 10000
Epoch: 0
Idx: 0 
Time for convolution layer by host (Sequential): 726.412 ms
Time for convolution layer by host (Sequential): 891.236 ms
Time for entire forward-pass: 1828.030 ms
Idx: 128 
Time for convolution layer by host (Sequential): 947.497 ms
Time for convolution layer by host (Sequential): 1316.995 ms
Time for entire forward-pass: 2546.461 ms
Idx: 256 
Time for convolution layer by host (Sequential): 1007.912 ms
Time for convolution layer by host (Sequential): 1298.071 ms
Time for entire forward-pass: 2599.945 ms
Idx: 384 
Time for convolution layer by host (Sequential): 766.548 ms
Time for convolution layer by host (Sequential): 873.146 ms
Time for entire forward-pass: 1840.250 ms
Idx: 512 
Time for convolution layer by host (Sequential): 708.534 ms
Time for convolution layer by host (Sequential): 899.711 ms
Time for entire forward-pass: 1824.130 ms
Idx: 640 
Time for convolution layer by host (Sequential): 745.404 ms
^C


In [7]:
!make clean

rm -f src/network.o src/mnist.o src/layer/fully_connected.o src/layer/max_pooling.o src/layer/relu.o src/layer/softmax.o src/layer/conv.o demo.o demo


#### 2.2. Cài đặt song song cơ bản

In [10]:
!make

gcc   -c -o src/network.o src/network.cc -Ithird_party/eigen
gcc   -c -o src/mnist.o src/mnist.cc -Ithird_party/eigen
gcc   -c -o src/layer/fully_connected.o src/layer/fully_connected.cc -Ithird_party/eigen
gcc   -c -o src/layer/max_pooling.o src/layer/max_pooling.cc -Ithird_party/eigen
gcc   -c -o src/layer/relu.o src/layer/relu.cc -Ithird_party/eigen
gcc   -c -o src/layer/softmax.o src/layer/softmax.cc -Ithird_party/eigen
nvcc   -c -o src/layer/conv.o src/layer/conv.cu -Ithird_party/eigen
third_party/eigen/Eigen/src/Core/DenseStorage.h(223): warning #20012-D: __host__ annotation is ignored on a function("DenseStorage") that is explicitly defaulted on its first declaration
      __attribute__((host)) __attribute__((device)) DenseStorage(const DenseStorage&) = default;
                     ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

third_party/eigen/Eigen/src/Core/DenseStorage.h(223): warning #20012-D: __device__ annotation is ignored on a functio

In [11]:
!./demo

mnist train number: 60000
mnist test number: 10000
Epoch: 0
Idx: 0 
Time for convolution layer by kernel 0 (Parallel): 874.963 ms
Time for convolution layer by kernel 0 (Parallel): 1223.462 ms
Time for entire forward-pass: 2389.571 ms
Idx: 128 
Time for convolution layer by kernel 0 (Parallel): 699.291 ms
Time for convolution layer by kernel 0 (Parallel): 1085.733 ms
Time for entire forward-pass: 2056.857 ms
Idx: 256 
Time for convolution layer by kernel 0 (Parallel): 493.406 ms
Time for convolution layer by kernel 0 (Parallel): 746.242 ms
Time for entire forward-pass: 1435.232 ms
Idx: 384 
Time for convolution layer by kernel 0 (Parallel): 488.886 ms
Time for convolution layer by kernel 0 (Parallel): 735.304 ms
Time for entire forward-pass: 1423.906 ms
Idx: 512 
Time for convolution layer by kernel 0 (Parallel): 490.740 ms
Time for convolution layer by kernel 0 (Parallel): 734.366 ms
Time for entire forward-pass: 1421.529 ms
Idx: 640 
Time for convolution layer by kernel 0 (Parallel):

In [9]:
!make clean

rm -f src/network.o src/mnist.o src/layer/fully_connected.o src/layer/max_pooling.o src/layer/relu.o src/layer/softmax.o src/layer/conv.o demo.o demo


#### 2.3. Cài đặt song song + Tối ưu

#### 2.3.1.

In [ ]:
!make

In [ ]:
!./demo

In [ ]:
!make clean

#### 2.3.2.

In [ ]:
!make

In [ ]:
!./demo

In [ ]:
!make clean